In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import pandas as pd 
import os
from datetime import datetime
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import seaborn as sns
from sklearn.metrics import mean_squared_log_error
import matplotlib.pyplot as plt
from collections import Counter
import math

In [2]:
data = pd.read_csv('train_0irEZ2H.csv')
target = np.log(data.units_sold)
data = data.drop(['units_sold'], axis = 1)

In [3]:
data.week = pd.to_datetime(data.week)
data['month'] = data['week'].dt.month
data['day'] = data['week'].dt.day
data['year'] = data['week'].dt.year
data['weeks'] = data['week'].dt.week
data['daysofweek'] = data['week'].dt.dayofweek
data['daysofyear'] = data['week'].dt.dayofyear
data.drop('week', axis =1, inplace= True)

In [30]:
a = pd.crosstab(index = data.store_id,columns=[data['sku_id']],values=data.is_featured_sku,aggfunc='sum')
a.fillna(0,inplace=True) 

In [32]:
a["b"] = a.sum(axis=1)
a =a.iloc[:,:28].div(a["b"], axis=0)

In [37]:
b= a.reset_index().melt(id_vars = 'store_id',value_name = 'prob')
data= data.merge(b,on= ['store_id','sku_id'])

In [38]:
np.corrcoef(data.prob,target)

array([[1.        , 0.00489691],
       [0.00489691, 1.        ]])

In [5]:
key = np.where(data.columns)[0].tolist()
values = list(data.columns.values)
dic = dict(zip(key,values))
dic

{0: 'record_ID',
 1: 'store_id',
 2: 'sku_id',
 3: 'total_price',
 4: 'base_price',
 5: 'is_featured_sku',
 6: 'is_display_sku',
 7: 'month',
 8: 'day',
 9: 'year',
 10: 'weeks',
 11: 'daysofweek',
 12: 'daysofyear'}

In [6]:
for i in [0,1,2,5,6,9]:
    data[dic[i]] =data[dic[i]].astype('object')
cat_var = np.where(data.dtypes == np.object)[0]

In [45]:
x_train,x_test,y_train,y_test = train_test_split(data.values,target.values,test_size = 0.25, random_state = 100)

In [ ]:
cat = CatBoostRegressor(iterations=5000)
model = cat.fit(x_train,y_train,cat_features = cat_var)

Learning rate set to 0.02663
0:	learn: 0.9002590	total: 223ms	remaining: 18m 35s
1:	learn: 0.9000177	total: 378ms	remaining: 15m 44s
2:	learn: 0.8998206	total: 605ms	remaining: 16m 47s
3:	learn: 0.8996297	total: 704ms	remaining: 14m 39s
4:	learn: 0.8994368	total: 803ms	remaining: 13m 22s
5:	learn: 0.8992380	total: 896ms	remaining: 12m 25s
6:	learn: 0.8990652	total: 1.01s	remaining: 12m
7:	learn: 0.8989456	total: 1.04s	remaining: 10m 52s
8:	learn: 0.8987773	total: 1.16s	remaining: 10m 44s
9:	learn: 0.8986189	total: 1.2s	remaining: 9m 59s
10:	learn: 0.8984735	total: 1.33s	remaining: 10m 2s
11:	learn: 0.8983239	total: 1.4s	remaining: 9m 41s
12:	learn: 0.8981837	total: 1.47s	remaining: 9m 24s
13:	learn: 0.8980404	total: 1.51s	remaining: 8m 58s
14:	learn: 0.8978976	total: 1.55s	remaining: 8m 35s
15:	learn: 0.8977791	total: 1.68s	remaining: 8m 42s
16:	learn: 0.8976534	total: 1.76s	remaining: 8m 35s
17:	learn: 0.8975380	total: 1.8s	remaining: 8m 18s
18:	learn: 0.8974246	total: 1.84s	remaining

158:	learn: 0.8932539	total: 16.1s	remaining: 8m 9s
159:	learn: 0.8932403	total: 16.3s	remaining: 8m 14s
160:	learn: 0.8932340	total: 16.5s	remaining: 8m 16s
161:	learn: 0.8932095	total: 16.8s	remaining: 8m 20s
162:	learn: 0.8932038	total: 17s	remaining: 8m 25s
163:	learn: 0.8931959	total: 17.1s	remaining: 8m 25s
164:	learn: 0.8931898	total: 17.3s	remaining: 8m 28s
165:	learn: 0.8931567	total: 17.7s	remaining: 8m 35s
166:	learn: 0.8931566	total: 17.8s	remaining: 8m 35s
167:	learn: 0.8931484	total: 18s	remaining: 8m 37s
168:	learn: 0.8931319	total: 18.2s	remaining: 8m 41s
169:	learn: 0.8931051	total: 18.5s	remaining: 8m 45s
170:	learn: 0.8931015	total: 18.6s	remaining: 8m 46s
171:	learn: 0.8930957	total: 18.7s	remaining: 8m 45s
172:	learn: 0.8930865	total: 18.9s	remaining: 8m 46s
173:	learn: 0.8930776	total: 19.1s	remaining: 8m 50s
174:	learn: 0.8930663	total: 19.4s	remaining: 8m 53s
175:	learn: 0.8930331	total: 19.6s	remaining: 8m 57s
176:	learn: 0.8930311	total: 19.7s	remaining: 8m 57

314:	learn: 0.8918855	total: 39.5s	remaining: 9m 47s
315:	learn: 0.8918804	total: 39.7s	remaining: 9m 47s
316:	learn: 0.8918771	total: 39.7s	remaining: 9m 46s
317:	learn: 0.8918669	total: 39.8s	remaining: 9m 46s
318:	learn: 0.8918666	total: 39.9s	remaining: 9m 45s
319:	learn: 0.8918636	total: 40s	remaining: 9m 44s
320:	learn: 0.8918582	total: 40s	remaining: 9m 43s
321:	learn: 0.8918582	total: 40s	remaining: 9m 41s
322:	learn: 0.8918562	total: 40.1s	remaining: 9m 40s
323:	learn: 0.8918555	total: 40.1s	remaining: 9m 39s
324:	learn: 0.8918551	total: 40.2s	remaining: 9m 37s
325:	learn: 0.8918468	total: 40.3s	remaining: 9m 37s
326:	learn: 0.8918287	total: 40.4s	remaining: 9m 37s
327:	learn: 0.8918083	total: 40.5s	remaining: 9m 37s
328:	learn: 0.8917952	total: 40.7s	remaining: 9m 37s
329:	learn: 0.8917841	total: 40.8s	remaining: 9m 37s
330:	learn: 0.8917727	total: 40.9s	remaining: 9m 37s
331:	learn: 0.8917606	total: 41s	remaining: 9m 36s
332:	learn: 0.8917565	total: 41.1s	remaining: 9m 36s
3

470:	learn: 0.8904942	total: 1m	remaining: 9m 45s
471:	learn: 0.8904939	total: 1m	remaining: 9m 44s
472:	learn: 0.8904867	total: 1m 1s	remaining: 9m 45s
473:	learn: 0.8904757	total: 1m 1s	remaining: 9m 45s
474:	learn: 0.8904589	total: 1m 1s	remaining: 9m 45s
475:	learn: 0.8904531	total: 1m 1s	remaining: 9m 45s
476:	learn: 0.8904370	total: 1m 1s	remaining: 9m 45s
477:	learn: 0.8904290	total: 1m 1s	remaining: 9m 45s
478:	learn: 0.8904219	total: 1m 2s	remaining: 9m 46s
479:	learn: 0.8904180	total: 1m 2s	remaining: 9m 45s
480:	learn: 0.8904008	total: 1m 2s	remaining: 9m 46s
481:	learn: 0.8903927	total: 1m 2s	remaining: 9m 45s
482:	learn: 0.8903924	total: 1m 2s	remaining: 9m 44s
483:	learn: 0.8903574	total: 1m 2s	remaining: 9m 45s
484:	learn: 0.8903455	total: 1m 2s	remaining: 9m 44s
485:	learn: 0.8903185	total: 1m 2s	remaining: 9m 44s
486:	learn: 0.8903087	total: 1m 3s	remaining: 9m 44s
487:	learn: 0.8903039	total: 1m 3s	remaining: 9m 44s
488:	learn: 0.8902912	total: 1m 3s	remaining: 9m 44s

624:	learn: 0.8887501	total: 1m 30s	remaining: 10m 30s
625:	learn: 0.8887471	total: 1m 30s	remaining: 10m 30s
626:	learn: 0.8887434	total: 1m 30s	remaining: 10m 29s
627:	learn: 0.8887336	total: 1m 30s	remaining: 10m 29s
628:	learn: 0.8887249	total: 1m 30s	remaining: 10m 28s
629:	learn: 0.8887142	total: 1m 30s	remaining: 10m 28s
630:	learn: 0.8887117	total: 1m 30s	remaining: 10m 28s
631:	learn: 0.8887071	total: 1m 30s	remaining: 10m 27s
632:	learn: 0.8886914	total: 1m 30s	remaining: 10m 27s
633:	learn: 0.8886839	total: 1m 31s	remaining: 10m 27s
634:	learn: 0.8886803	total: 1m 31s	remaining: 10m 26s
635:	learn: 0.8886708	total: 1m 31s	remaining: 10m 26s
636:	learn: 0.8886650	total: 1m 31s	remaining: 10m 25s
637:	learn: 0.8886612	total: 1m 31s	remaining: 10m 25s
638:	learn: 0.8886597	total: 1m 31s	remaining: 10m 24s
639:	learn: 0.8886594	total: 1m 31s	remaining: 10m 23s
640:	learn: 0.8886538	total: 1m 31s	remaining: 10m 23s
641:	learn: 0.8886517	total: 1m 31s	remaining: 10m 22s
642:	learn

In [47]:
y_pred = model.predict(x_train)
print(math.sqrt(mean_squared_log_error(np.exp(y_train),np.exp(y_pred)))*100)
y_pred = model.predict(x_test)
print(math.sqrt(mean_squared_log_error(np.exp(y_test),np.exp(y_pred)))*100)

83.81062239365507
85.4161082131994


In [62]:
pd.DataFrame(model.feature_importances_,data.columns)

,0
record_ID,0.000000
store_id,22.658855
sku_id,25.897386
total_price,14.076932
base_price,5.503304
is_featured_sku,9.806019
is_display_sku,7.554258
month,0.725852
day,4.854576
year,2.129989


In [49]:
test = pd.read_csv('test_nfaJ3J5.csv')
#test = test.drop(['record_ID'], axis =1)

In [50]:
test.week = pd.to_datetime(test.week)
test['month'] = test['week'].dt.month
test['day'] = test['week'].dt.day
test['year'] = test['week'].dt.year
test['weeks'] = test['week'].dt.week
test['dayofweek'] = test['week'].dt.dayofweek
test['daysofyear'] = test['week'].dt.dayofyear
test.drop('week', axis =1, inplace= True)

In [51]:
test.head()

,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,month,day,year,weeks,dayofweek,daysofyear
0,212645,8091,216418,108.3000,108.3000,0,0,7,16,2013,29,1,197
1,212646,8091,216419,109.0125,109.0125,0,0,7,16,2013,29,1,197
2,212647,8091,216425,133.9500,133.9500,0,0,7,16,2013,29,1,197
3,212648,8091,216233,133.9500,133.9500,0,0,7,16,2013,29,1,197
4,212649,8091,217390,176.7000,176.7000,0,0,7,16,2013,29,1,197


In [52]:
for i in [0,1,2,5,6,9]:
    test[dic[i]] =test[dic[i]].astype('object')

In [57]:
y_pred = model.predict(test.values)
y_pred =np.round(y_pred)

In [58]:
sample =pd.read_csv('sample_submission_pzljTaX.csv')

In [59]:
final_result = pd.DataFrame({'record_ID':sample.record_ID.values,"units_sold":y_pred},dtype=np.object)
final_result
final_result.to_csv('Final_Submission.csv',index=False)

In [60]:
final_result

,record_ID,units_sold
0,212645,3
1,212646,3
2,212647,3
3,212648,3
4,212649,3
...,...,...
13855,232281,3
13856,232282,3
13857,232285,3
13858,232286,2
